## SRJ 

In [2]:
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
from scipy.optimize import minimize
ash_excitation = []
energies = []
excitations= []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(symbols, coordinates, active_electrons, active_orbitals, adapt_it, shots = None):
    print('Using active space, check if you change the H accordingly')
    #H, qubits = qml.qchem.molecular_hamiltonian(symbols, coordinates, basis="sto-3g", method="pyscf",active_electrons=active_electrons, active_orbitals=active_orbitals)
    core, active = qml.qchem.active_space(electrons, orbitals, active_electrons=active_electrons, active_orbitals=active_orbitals) # Spatial orbitals
    print('core orbitals:', core)
    print('active orbitals:', active)
    #print(H)
    return core, active

In [4]:
symbols  = [ 'Fe', 'S']
print('H2O-0.6A-GS+ES-DBS')
r_bohr = 1.8897259886 
coordinates = np.array([[0.0,0.0, 0.0], [0.0, 0.0, 1.826*r_bohr]])

print(coordinates)

electrons = 42  # 23 from N and 3 from H
orbitals = 54 #Spin orbitals  
charge = 0

active_electrons = 6 #No of electrons
active_orbitals = 6 #Spatial orbitals

core, active = ags_exact(symbols, coordinates, active_electrons, active_orbitals, shots = None, adapt_it=1)

H2O-0.6A-GS+ES-DBS
[[0.         0.         0.        ]
 [0.         0.         3.45063966]]
Using active space, check if you change the H accordingly
core orbitals: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
active orbitals: [18, 19, 20, 21, 22, 23]


In [7]:
import basis_set_exchange as bse

basis_dict = bse.get_basis('ANO-RCC-MB', elements=['Fe', 'S'], fmt='nwchem')

import numpy as np
from pyscf import gto, scf, mcscf
from pyscf import gto, ao2mo, scf
import pennylane as qml

# Your original setup (unchanged) 
mol_pyscf = gto.M(
    atom = 'Fe 0 0 0; S 0 0 1.826',
    basis = basis_dict,
    spin = 4)
rhf = scf.ROHF(mol_pyscf)
rhf.kernel()

import pennylane as qml 
from pennylane import qchem


one_ao = mol_pyscf.intor_symmetric("int1e_kin") + mol_pyscf.intor_symmetric("int1e_nuc")
two_ao = mol_pyscf.intor("int2e_sph")


one_mo = np.einsum("pi,pq,qj->ij", rhf.mo_coeff, one_ao, rhf.mo_coeff, optimize=True)
two_mo = ao2mo.incore.full(two_ao, rhf.mo_coeff)

core_constant = np.array([rhf.energy_nuc()])
print('The value of core:', core_constant)

# convert the two-electron integral tensor to the physicists’ notation
two_mo = np.swapaxes(two_mo, 1, 3)


core=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17] 
active=[18, 19, 20, 21, 22, 23]

total_orb = len(core) + len(active)
print('Total orbitals are',total_orb)

# define the active space and recompute the integrals
def pyscf_integral(core, active, one_mo, two_mo, core_constant):
    if core and active:
        for i in core:
            core_constant = core_constant + 2 * one_mo[i][i]
            for j in core:
                core_constant = core_constant + 2 * two_mo[i][j][j][i] - two_mo[i][j][i][j]

        for p in active:
            for q in active:
                for i in core:
                    print(f'The value of p and q', {p}, {q})
                    one_mo[p, q] = one_mo[p, q] + (2 * two_mo[i][p][q][i] - two_mo[i][p][i][q])

        one_mo = one_mo[qml.math.ix_(active, active)]
        two_mo = two_mo[qml.math.ix_(active, active, active, active)]

    return core_constant, one_mo, two_mo

core_constant, one_mo, two_mo = pyscf_integral(core, active, one_mo,two_mo, core_constant)

H_fermionic = qml.qchem.fermionic_observable(core_constant, one_mo, two_mo)

H = qml.jordan_wigner(H_fermionic)
print(H)

converged SCF energy = -1655.34968383623
The value of core: [120.55734926]
Total orbitals are 24
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {18}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The value of p and q {18} {19}
The 

In [8]:
qubits = 12
active_electrons = 6
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("lightning.qubit", wires = qubits)
@qml.qnode(dev)
def e():
    qml.BasisState(hf_state, wires=range(qubits))
    return qml.expval(H)

fe = e()
print('fe', fe)

fe -1655.1633906935663


1.  Energy produced by H should match the ROHF. 

# Using pennylane format - HF state

In [2]:




#Check the possible excitations
import pennylane as qml
from pennylane import qchem
from pennylane import numpy as np
from itertools import chain
import time
import re
import warnings
#warnings.filterwarnings("ignore", category=np.ComplexWarning)
from scipy.optimize import minimize
optimizer = qml.AdamOptimizer(stepsize=0.5)
ash_excitation = []
energies = []
excitations= []
old_grad = []

X = qml.PauliX
Y = qml.PauliY
Z = qml.PauliZ
I = qml.Identity






def ags_exact(active_electrons, H, adapt_it):
    print('Using active space, check if you change the H accordingly')
    qubits = 12
    active_electrons = active_electrons
    #print(H)
    hf_state = qchem.hf_state(active_electrons, qubits)
    #hf_state = np.array([1,1,1,0,0,0,1,1,1,0,0,0])
    #Calculation of HF state
    dev = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev)
    def circuit(hf_state, active_electrons, qubits, H): 
        qml.BasisState(hf_state, wires=range(qubits))
        return qml.expval(H)   #Calculating the expectation value of the Hamiltonian
    
    # Commutator calculation for HF state
    @qml.qnode(dev)
    def commutator_0(H,w, k):  #H is the Hamiltonian, w is the operator, k is the basis state - HF state
        qml.BasisState(k, wires=range(qubits))
        res = qml.commutator(H, w)   #Calculating the commutator
        return qml.expval(res)
    
    # Commutator calculation for other states except HF state
    @qml.qnode(dev)
    def commutator_1(H,w, k): #H is the Hamiltonian, w is the operator, k is the basis state
        qml.StatePrep(k, wires=range(qubits))
        res = qml.commutator(H, w) #Calculating the commutator
        return qml.expval(res)

    #Energy calculation 
    @qml.qnode(dev)
    def ash(params, ash_excitation, hf_state, H):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]  #Appln of HF state
        for i, excitation in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.expval(H)  #Calculating the expectation value of the Hamiltonian
    
    # Calculation of New state, same as the above function but with the state return
    dev1 = qml.device("lightning.qubit", wires=qubits)
    @qml.qnode(dev1)
    def new_state(hf_state, ash_excitation, params):
        [qml.PauliX(i) for i in np.nonzero(hf_state)[0]] #Applying the HF state
        for i, excitations in enumerate(ash_excitation):
            if len(ash_excitation[i]) == 4:
                qml.FermionicDoubleExcitation(weight=params[i], wires1=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)), wires2=list(range(ash_excitation[i][2], ash_excitation[i][3] + 1)))
            elif len(ash_excitation[i]) == 2:
                qml.FermionicSingleExcitation(weight=params[i], wires=list(range(ash_excitation[i][0], ash_excitation[i][1] + 1)))
        return qml.state()
    

    
    
    def cost(params):
        energy = ash(params, ash_excitation, hf_state, H)
        return energy

    #def callback(params):
        #print(f"Current parameters: {params}")
        #print(f"Current cost: {cost(params)}\n")
    

    print('HF state is', circuit(hf_state, active_electrons, qubits, H))
    singles, doubles = qml.qchem.excitations(active_electrons, qubits)

    op1 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "-"}) for x in singles]
    op2 =  [qml.fermi.FermiWord({(0, x[0]): "+", (1, x[1]): "+", (2, x[2]): "-", (3, x[3]): "-"})for x in doubles]
    operator_pool = (op1) + (op2)  #Operator pool - Singles and Doubles
    print('Total excitations are', len(operator_pool))
    states = [hf_state]
    params = np.zeros(len(ash_excitation), requires_grad=True) 


    

    for j in range(adapt_it):
        print('The adapt iteration now is', j)  #Adapt iteration
        max_value = float('-inf')
        max_operator = None
        k = states[-1] if states else hf_state  # if states is empty, fall back to hf_state
       
        for i in operator_pool:
            #print('The current excitation operator is', i)   #Current excitation operator - fermionic one
            w = qml.fermi.jordan_wigner(i)  #JW transformation
            if np.array_equal(k, hf_state): # If the current state is the HF state
                current_value = abs(2*(commutator_0(H, w, k)))      #Commutator calculation is activated  
            else:
                current_value = abs(2*(commutator_1(H, w, k)))      #For other states, commutator calculation is activated

            if current_value > max_value:
                max_value = current_value
                max_operator = i

        print(f"The highest operator value is {max_value} for operator {max_operator}")  #Highest operator value

        old_grad.append(max_value)  #Appending the old gradient value
        indices_str = re.findall(r'\d+', str(max_operator))
        excitations = [int(index) for index in indices_str]
        print('Highest gradient excitation is', excitations)
        ash_excitation.append(excitations) #Appending the excitations to the ash_excitation

        params = np.append(params, 0.0)  #Parameters initialization

        #Energy calculation
        result = minimize(cost, params, method='BFGS', tol = 1e-16, options = {'disp': False, 'maxiter': 1e8, 'gtol': 1e-12})

        print("Final updated parameters:", result.x)
        print("Final cost:", result.fun)

        params= (result.x)
        energies.append(result.fun)



        ostate = new_state(hf_state, ash_excitation, params)
        print(qml.draw(new_state, max_length=100)(hf_state,ash_excitation,params))
        gs_state = ostate
        states.append(ostate)
        
    return gs_state, params, ash_excitation, qubits, H, energies, old_grad






#electrons = 10  # 7 from N and 3 from H
#orbitals = 16
charge = 0

active_electrons = 6


gs_state, params, ash_excitation, qubits, H,energies, old_grad = ags_exact( active_electrons, H, adapt_it=50) #1 is used for params



print('The params after GS is',params)
print('Ash excitation after gs state:', ash_excitation)
print('Energies:', energies)
print('Old gradient:', old_grad)    


Using active space, check if you change the H accordingly
HF state is -1655.1633906935663
Total excitations are 117
The adapt iteration now is 0


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 0.10812497583517312 for operator a⁺(0) a⁺(1) a(10) a(11)
Highest gradient excitation is [0, 1, 10, 11]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [0.23483423]
Final cost: -1655.1697678320475
 0: ──X─╭FermionicDoubleExcitation(0.23)─┤  State
 1: ──X─├FermionicDoubleExcitation(0.23)─┤  State
 2: ──X─│────────────────────────────────┤  State
 3: ──X─│────────────────────────────────┤  State
 4: ──X─│────────────────────────────────┤  State
 5: ──X─│────────────────────────────────┤  State
10: ────├FermionicDoubleExcitation(0.23)─┤  State
11: ────╰FermionicDoubleExcitation(0.23)─┤  State
The adapt iteration now is 1


...

Final updated parameters: [ 3.07206853e-01 -1.76613486e-05 -5.92732873e-04  6.16080429e-01
  6.14508731e-01 -1.18569065e-01 -1.57126919e+00  5.08498188e-04
 -1.17594725e-01 -3.14517133e+00 -6.22425852e-05  1.24026091e-05
  6.34014030e-04  7.78480814e-04  9.72234954e-04  4.92978507e-04
  6.42346449e-04 -8.97863519e-06  1.35677183e-04 -1.48592126e-04
 -4.03468465e-05 -2.99050777e-05 -2.92113060e-05 -1.46891739e-04
 -1.48811099e-04 -3.90380228e-04 -5.73969934e-05 -5.87402714e-05
  1.06812676e-04  3.13396308e-14  3.32460436e-18  7.07879596e-23
 -1.41575919e-22  0.00000000e+00 -7.07879596e-23  2.35959865e-23
  2.35959865e-23 -7.07879596e-23 -2.35959865e-23  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
Final cost: -1655.308520883909
 0: ──X─╭FermionicDoubleExcitation(0.31)────────────────────────────────── ···
 1: ──X─├FermionicDoubleExcitation(0.31)──────────────────────────

/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


The highest operator value is 5.9879398329295655e-05 for operator a⁺(3) a⁺(4) a(6) a(9)
Highest gradient excitation is [3, 4, 6, 9]


/Users/s.poyyapakkam/miniconda3/envs/na/lib/python3.11/site-packages/pennylane_lightning/lightning_base/_serialize.py:359: ComplexWarning: Casting complex values to real discards the imaginary part
  coeffs = np.array(coeffs).astype(self.rtype)


Final updated parameters: [ 3.07206853e-01 -1.76613486e-05 -5.92732873e-04  6.16080429e-01
  6.14508731e-01 -1.18569065e-01 -1.57126919e+00  5.08498188e-04
 -1.17594725e-01 -3.14517133e+00 -6.22425852e-05  1.24026091e-05
  6.34014030e-04  7.78480814e-04  9.72234954e-04  4.92978507e-04
  6.42346449e-04 -8.97863519e-06  1.35677183e-04 -1.48592126e-04
 -4.03468465e-05 -2.99050777e-05 -2.92113060e-05 -1.46891739e-04
 -1.48811099e-04 -3.90380228e-04 -5.73969934e-05 -5.87402714e-05
  1.06812676e-04  3.13396308e-14  3.32460436e-18  7.07879596e-23
 -1.41575919e-22  0.00000000e+00 -7.07879596e-23  2.35959865e-23
  2.35959865e-23 -7.07879596e-23 -2.35959865e-23  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00]
Final cost: -1655.3085208838977
 0: ──X─╭FermionicDoubleExcitation(0.31)────────────────────────────────── ···
 1: ──X─├FermionicDoubleExcitation(0.31)─────────

In [3]:
energies

[-1655.1697678320475,
 -1655.193248120223,
 -1655.2057005390784,
 -1655.2432085677262,
 -1655.246586348109,
 -1655.2473051516004,
 -1655.2770120048424,
 -1655.2772736485945,
 -1655.2775881515895,
 -1655.3085193509735,
 -1655.3085205452417,
 -1655.3085205452312,
 -1655.3085205452207,
 -1655.3085206323292,
 -1655.3085207398753,
 -1655.308520739864,
 -1655.3085207703803,
 -1655.3085207777965,
 -1655.3085207955514,
 -1655.308520795598,
 -1655.308520798619,
 -1655.3085207986094,
 -1655.3085207985991,
 -1655.308520798589,
 -1655.3085207985787,
 -1655.3085207985675,
 -1655.3085207986085,
 -1655.3085207991637,
 -1655.308520884106,
 -1655.3085208841026,
 -1655.3085208840926,
 -1655.308520884083,
 -1655.3085208840728,
 -1655.3085208840625,
 -1655.3085208840525,
 -1655.308520884043,
 -1655.3085208840328,
 -1655.3085208840223,
 -1655.3085208840118,
 -1655.3085208840014,
 -1655.3085208839912,
 -1655.3085208839805,
 -1655.3085208839702,
 -1655.3085208839589,
 -1655.3085208839493,
 -1655.308520883939